In [ ]:
import torch
from torch import nn
from torch.utils.data import DataLoader
from torchvision import datasets, transforms
import torch.nn.functional as F

transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.1307,), (0.3081,))
])

train_data = datasets.MNIST(root="data", train=True, download=True, transform=transform)
test_data = datasets.MNIST(root="data", train=False, download=True, transform=transform)

train_loader = DataLoader(train_data, batch_size=128, shuffle=True)
test_loader = DataLoader(test_data, batch_size=128, shuffle=False)

class CNN(nn.Module):
    def __init__(self):
        super().__init__()
        self.conv1 = nn.Conv2d(1, 64, kernel_size=3, padding=1) #
        self.conv2 = nn.Conv2d(64, 128, kernel_size=3, padding=1)
        
        self.pool = nn.MaxPool2d(2, 2)
        self.dropout1 = nn.Dropout(0.25)
        self.dropout2 = nn.Dropout(0.5)
        
        self.fc1 = nn.Linear(128 * 7 * 7, 512)
        self.fc2 = nn.Linear(512, 512)
        self.fc3 = nn.Linear(512, 10)
        
        self.bn1 = nn.BatchNorm2d(64)
        self.bn2 = nn.BatchNorm2d(128)

    def forward(self, x):
        x = self.pool(F.relu(self.bn1(self.conv1(x)))) #1, 28, 28 -> 64, 28, 28 -> 64, 14, 14
        x = self.pool(F.relu(self.bn2(self.conv2(x)))) #64, 14, 14 -> 128, 14, 14 -> 128, 7, 7
        x = x.view(x.size(0), -1)
        x = F.relu(self.fc1(x))
        x = self.dropout2(x)
        x = F.relu(self.fc2(x))
        x = self.fc3(x)
        return x

model = CNN()

loss_fn = nn.CrossEntropyLoss()
optimizer = torch.optim.AdamW(model.parameters(), lr=0.001, weight_decay=1e-4)

def train(dataloader, model, loss_fn, optimizer):
    model.train()
    for batch, (X, y) in enumerate(dataloader):
        pred = model(X)
        loss = loss_fn(pred, y)
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        
        if batch % 100 == 0:
            print(f"loss: {loss.item():>7f} [{batch*len(X):>5d}/{len(dataloader.dataset):>5d}]")

def test(dataloader, model, loss_fn):
    size = len(dataloader.dataset)
    model.eval()
    test_loss, correct = 0, 0
    
    with torch.no_grad():
        for X, y in dataloader:
            pred = model(X)
            test_loss += loss_fn(pred, y).item()
            correct += (pred.argmax(1) == y).type(torch.float).sum().item()
    
    accuracy = correct / size
    print(f"Accuracy: {(100*accuracy):>0.1f}%\n")

epochs = 3
for t in range(epochs):
    print(f"Epoch {t+1}\n-------------------------------")
    train(train_loader, model, loss_fn, optimizer)
    test(test_loader, model, loss_fn)

print("Done!")
torch.save(model.state_dict(), 'mnist_cnn.pth')

import torch.onnx

model.eval()


dummy_input = torch.randn(1, 1, 28, 28)

# Export en ONNX
torch.onnx.export(
    model,                         
    dummy_input,
    "mnist_cnn_web.onnx",          
    export_params=True,            
    opset_version=11,              
    do_constant_folding=True,      
    input_names=['input'],
    output_names=['output'],       
    dynamic_axes={                 
        'input': {0: 'batch_size'},
        'output': {0: 'batch_size'}
    }
)

print("✅ Modèle exporté vers mnist_cnn_web.onnx")

Epoch 1
-------------------------------
loss: 2.296385 [    0/60000]
loss: 0.204428 [12800/60000]
loss: 0.135629 [25600/60000]
loss: 0.080116 [38400/60000]
loss: 0.078821 [51200/60000]
Accuracy: 98.3%

Epoch 2
-------------------------------
loss: 0.126362 [    0/60000]
loss: 0.063290 [12800/60000]
loss: 0.061931 [25600/60000]
loss: 0.143564 [38400/60000]
loss: 0.080288 [51200/60000]
Accuracy: 98.9%

Epoch 3
-------------------------------
loss: 0.077238 [    0/60000]
loss: 0.162501 [12800/60000]
loss: 0.086075 [25600/60000]
loss: 0.045436 [38400/60000]
loss: 0.028778 [51200/60000]
Accuracy: 98.9%

Done!
✅ Modèle exporté vers mnist_cnn_web.onnx
